# 参数管理

- 依靠深度学习框架管理操作参数：

    * 访问参数，用于调试、诊断和可视化
    * 参数初始化
    * 在不同模型组件间共享参数

考虑一个单隐藏层的神经网络

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0379],
        [0.0994]], grad_fn=<AddmmBackward0>)

## 参数访问

- 当通过`Sequential`类定义模型时，可以通过索引来访问模型的任意层
    - 这就像模型是一个列表一样，每层的参数都在其属性中

In [2]:
print(f'检查第二个全连接层的参数\n{net[2].state_dict()}')

检查第二个全连接层的参数
OrderedDict([('weight', tensor([[ 0.2658, -0.2868, -0.0546,  0.2135,  0.1780,  0.0741, -0.0212, -0.0069]])), ('bias', tensor([-0.0393]))])


- 这个全连接层包含两个参数，分别是该层的权重和偏置
    - 参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此

- `state_dict()`是一个`Python`字典，将每层映射到对应的参数。一般下述对象拥有`state_dict`
    - 包含可学习参数的层
    - 优化器`torch.optim`

### 访问目标参数

- 每个参数都表示为参数类的一个实例

In [4]:
# 下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，提取后返回的是一个参数类实例，并进一步访问该参数的值。
print(type(net[2].bias))
print('================')
print(net[2].bias)
print('================')
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0393], requires_grad=True)
tensor([-0.0393])


- 参数是复合的对象，包含值、梯度和额外信息

In [6]:
# 在上面这个网络中，由于还没有调用反向传播，所以参数的梯度处于初始状态
net[2].weight.grad == None

True

### 一次性访问所有参数

- 递归整个树提取每个子块的参数

In [7]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print('=============================')
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


- `named_parameters()`返回一个遍历模型块的参数的迭代器，包含参数的名称与参数本身

- 这提供了另一种访问网络参数的方式


In [8]:
net.state_dict()['2.bias'].data

tensor([-0.0393])

### 从嵌套块收集参数

- 观察在多个块相互嵌套中参数如何命名？

In [10]:
# 定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中

def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.0277],
        [-0.0277]], grad_fn=<AddmmBackward0>)

In [11]:
print(rgnet) # 观察嵌套块结构和参数命名

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


- 因为层是分层嵌套的，所以可以像通过嵌套列表索引一样访问它们

In [12]:
rgnet[0][1][0].bias.data # 访问第一个主要的块中、第二个子块的第一层的偏置项

tensor([-0.0941,  0.4905,  0.0289, -0.3920,  0.0739, -0.2282, -0.4643,  0.0487])

## 参数初始化

- 默认情况下，`PyTorch`会根据一个范围均匀地初始化权重和偏置矩阵
- `PyTorch`的`nn.init`模块提供了多种预置初始化方法。

### 内置初始化

In [15]:
# 下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，且将偏置参数设置为0

def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
print(f'权重为\n{net[0].weight.data}\n偏置为\n{net[0].bias.data}')

权重为
tensor([[ 0.0020, -0.0077, -0.0098,  0.0030],
        [-0.0077,  0.0005,  0.0021, -0.0037],
        [-0.0069, -0.0029,  0.0204, -0.0100],
        [ 0.0166, -0.0042,  0.0133,  0.0012],
        [-0.0089, -0.0059, -0.0025, -0.0075],
        [-0.0233,  0.0114,  0.0035, -0.0037],
        [ 0.0013,  0.0044,  0.0047,  0.0061],
        [-0.0107,  0.0013,  0.0014,  0.0088]])
偏置为
tensor([0., 0., 0., 0., 0., 0., 0., 0.])


- ```python
torch.nn.Module.apply(fn)
```

    - 将函数`fn`递归应用到**块的自身**和包含的**每个子块**

In [16]:
# 还可以将所有参数初始化为给定的常数，比如初始化为1

def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
print(f'权重为\n{net[0].weight.data}\n偏置为\n{net[0].bias.data}')

权重为
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
偏置为
tensor([0., 0., 0., 0., 0., 0., 0., 0.])


- 对某些块应用不同的初始化方法

In [17]:
# 下面使用Xavier初始化方法初始化第一个神经网络层，然后将第三个神经网络层初始化为常量值42

def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)

print(f'块0的权重为\n{net[0].weight.data}')
print(f'块2的权重为\n{net[2].weight.data}')

块0的权重为
tensor([[-0.4273,  0.6390, -0.6410, -0.5123],
        [ 0.6606, -0.5980, -0.2207, -0.5729],
        [-0.4060, -0.0907, -0.0291, -0.0865],
        [ 0.0791, -0.0904, -0.5871,  0.1705],
        [ 0.2206,  0.3919, -0.5668,  0.5917],
        [ 0.3217, -0.3232, -0.0541, -0.6284],
        [-0.0609, -0.2158,  0.0890,  0.2609],
        [ 0.6845,  0.4896,  0.4156,  0.6356]])
块2的权重为
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

- 应对深度学习框架没有提供需要的初始化方法的情形


- 下面的例子使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


In [20]:
# 自定义一个`my_init`函数来应用到`net`

def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)        # 用[-10,10]均匀分布初始化权重
        m.weight.data *= m.weight.data.abs() >= 5  # 绝对值大于5保留，否则等于0

net.apply(my_init)
print('=================')
print(f'初始化后的权重为\n{net[0].weight}')

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
初始化后的权重为
Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000, -8.6178],
        [ 0.0000, -7.1437, -7.6337,  9.6238],
        [-0.0000,  0.0000, -0.0000,  8.8234],
        [-8.9188, -0.0000,  5.5878,  0.0000],
        [-7.3273, -0.0000, -7.6454, -7.2279],
        [ 6.0692, -9.3481, -0.0000,  8.2564],
        [-9.0458,  5.3331,  0.0000,  0.0000],
        [ 6.2911, -0.0000,  8.2073, -6.7869]], requires_grad=True)


- 注意，我们始终可以直接设置参数

In [21]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000, -7.6178])

## 参数绑定

- 希望在多个层间共享参数：
    - 可以定义一个稠密层，然后使用它的参数来设置另一个层的参数

In [22]:
# 需要给共享层一个名称，以便可以引用它的参数

shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)

tensor([[0.1884],
        [0.1784]], grad_fn=<AddmmBackward0>)

In [23]:
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [24]:
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


- 上面例子表明第三个和第五个神经网络层的参数是绑定的，它们不仅值相等，而且由相同的张量表示。
- 因此，如果我们改变其中一个参数，另一个参数也会改变